In [ ]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
import sqlite3

#Kết nối vào db
connect = sqlite3.connect('Database.db')
c = connect.cursor()

#Tạo bảng mới
c.execute('''CREATE TABLE IF NOT EXISTS products (
          product_id INTEGER PRIMARY KEY,
          product_name TEXT NOT NULL,
          product_price REAL NOT NULL,
          product_stock INTEGER NOT NULL
          )''')

c.execute('''CREATE TABLE IF NOT EXISTS sales (
          sale_id INTEGER PRIMARY KEY,
          sale_date TEXT NOT NULL,
          sale_amount REAL NOT NULL,
          sale_items INTEGER NOT NULL
          )''')

#Tạo giao diện chính
window = tk.Tk()
window.title('SALES MANAGEMENT SYSTEM')
window.geometry('600x500')

#Tạo notebook widget 
notebook = ttk.Notebook(window)
notebook.pack(fill=BOTH, expand=True)

#Tạo Frame cho mỗi tab
register_sale_frame = ttk.Frame(notebook)
register_product_frame = ttk.Frame(notebook)
query_frame = ttk.Frame(notebook)

#Add frame vào notebook
notebook.add(register_sale_frame, text= 'Register Sale')
notebook.add(register_product_frame, text= 'Register Product')
notebook.add(query_frame, text= 'Query')

#Tạo các biến để lưu trữ sản phẩm
product_id_var = tk.StringVar()
product_name_var = tk.StringVar()
product_price_var = tk.IntVar()
product_stock_var = tk.IntVar()

#tạo các hàm để xử lý
def add_product():
    id = id_entry.get()
    name = name_entry.get()
    price = price_entry.get()
    stock = stock_entry.get()
    if id and name and price and stock:
        c.execute('INSERT INTO products VALUES (?,?,?,?)', (id, name, price, stock))
        connect.commit()
        messagebox.showinfo('Success', f'You have added {name} with price {price} and stock {stock}')

        id_entry.delete(0, 'end')
        name_entry.delete(0, 'end')
        price_entry.delete(0, 'end')
        stock_entry.delete(0, 'end')
        
        show_product()
        show()
    else:
        messagebox.showerror('Error', 'Please enter all fields')

def update():
    id = id_entry.get()
    name = name_entry.get()
    price = price_entry.get()
    stock = stock_entry.get()
    if id and name and price and stock:
        c.execute('UPDATE products SET product_name = ?, product_price = ?, product_stock = ? WHERE product_id = ?', (name, price, stock, id))
        connect.commit()
        messagebox.showinfo('Success', f'You have updated product with ID {id}')
        id_entry.delete(0, 'end')
        name_entry.delete(0, 'end')
        price_entry.delete(0, 'end')
        stock_entry.delete(0, 'end')
        show()
    else:
        messagebox.showerror('Error', 'Please fill all requirement')

def delete():
    id = id_entry.get()
    if id:
        c.execute('DELETE FROM products WHERE product_id = ?', (id,))
        connect.commit()
        messagebox.showinfo('Success', f'You have delete product with ID {id}')
        id_entry.delete(0, 'end')
        name_entry.delete(0, 'end')
        price_entry.delete(0, 'end')
        stock_entry.delete(0, 'end')
        show()
    else:
        messagebox.showerror('Error', 'Please enter the Product ID')

def query_inventory():
    query_type = query_combobox.get()
    if query_type:
        query_listbox.delete(0, 'end')
        if query_type == 'LIST ALL PRODUCT':
            c.execute('SELECT * FROM products')
        elif query_type == 'LIST PRODUCT THAT ARE OUT OF STOCK':
            c.execute('SELECT * FROM products WHERE product_stock = 0')
        elif query_type == 'LIST PRODUCT THAT ARE IN STOCK':
            c.execute('SELECT * FROM products WHERE product_stock > 0')
        elif query_type == 'LIST SALE BY DATE':
            c.execute("SELECT sale_date, SUM(sale_amount), SUM(sale_items) FROM sales GROUP BY sale_date")
            results = c.fetchall()
        if results:
            for result in results:
                query_listbox.insert('end', result)
        else:
            query_listbox.insert('end', 'No ressult found')
    else:
        messagebox.showerror('Error', 'Please select a query type')

def register_sale():
    selection = product_listbox.curselection()
    if selection:
        product_id = product_listbox.get(selection[0])[0]
        c.execute('SELECT * FROM products WHERE product_id = ?', (product_id,))
        product = c.fetchone()
        if product[3] > 0:
            new_stock = product[3] - 1
            c.execute('UPDATE products SET product_stock = ? WHERE product_id = ?', (new_stock, product_id))
            connect.commit()

            c.execute('INSERT INTO sales (sale_date, sale_amount, sale_items) VALUES (date("now"), ?, 1)', (product[2],))
            connect.commit()
            messagebox.showinfo('Success', f'You have sold {product[1]} for ${product[2]}')
            show_product()
        else:
            messagebox.showerror('Error', f'{product[1]} is out of stock')
    else:
        messagebox.showerror('Error', 'Please select a product to sell')

def selection(event):
    row = table.focus()
    info = table.item(row)
    values = info['values']

    product_id_var.set([0])
    product_name_var.set([1])
    product_price_var.set([2])
    product_stock_var.set([3])

def show():
    table.delete(*table.get_children())
    c.execute('SELECT * FROM products')
    prorducts_ = c.fetchall()
    for products in prorducts_:
        table.insert('', 'end', value=products)

def show_product():
    product_listbox.delete(0, 'end')
    c.execute('SELECT * FROM products')
    products = c.fetchall()
    for product in products:
        product_listbox.insert('end', product)

#Tạo một bảng để hiển thị danh sach sản phẩm
table = ttk.Treeview(register_product_frame)
table['columns'] = ('Product ID', 'Product name', 'Product price', 'Stock')
table.column('#0', width=0)
table.column('Product ID', width=100)
table.column('Product name', width=200)
table.column('Product price', width=100)
table.column('Stock', width=100)

table.heading('Product ID', text='Product ID')
table.heading('Product name', text='Product Name')
table.heading('Product price', text='Product Price')
table.heading('Stock', text='Stock')

table.grid(row=5, columnspan=2)

#Tạo các Widget cho trang bán hàng
product_listbox = tk.Listbox(register_sale_frame, width=100, height = 20)
product_listbox.grid(row=2, column=0, padx=10, pady=10)

register_sale_button = tk.Button(register_sale_frame, text='Register Sale', command=register_sale)
register_sale_button.grid(row=0, column=0, padx=10, pady=10)


#Tạo các Widget cho trang thêm thông tin sản phẩm
update_button = tk.Button(register_product_frame, text='Update', command=update)
update_button.grid(row=1, column=2, padx=10, pady=10)

delete_button = tk.Button(register_product_frame, text='Delete', command = delete)
delete_button.grid(row=2, column=2, padx=10, pady=10)

id_label = tk.Label(register_product_frame, text='Product ID')
id_label.grid(row=0, column=0, padx=10, pady = 10)

id_entry = tk.Entry(register_product_frame)
id_entry.grid(row=0, column=1, padx=10, pady = 10)

show_button = tk.Button(register_product_frame, text='Show List', command=show)
show_button.grid(row=3, column=2, padx=10, pady=10)

name_label = tk.Label(register_product_frame, text='Product Name')
name_label.grid(row=1, column=0, padx=10, pady = 10)

name_entry = tk.Entry(register_product_frame)
name_entry.grid(row=1, column=1, padx=10, pady = 10)

price_label = tk.Label(register_product_frame, text='Product Price')
price_label.grid(row=2, column=0, padx=10, pady = 10)

price_entry = tk.Entry(register_product_frame)
price_entry.grid(row=2, column=1, padx=10, pady = 10)

stock_label = tk.Label(register_product_frame, text='Product Stock')
stock_label.grid(row=3, column=0, padx=10, pady = 10)

stock_entry = tk.Entry(register_product_frame)
stock_entry.grid(row=3, column=1, padx=10, pady = 10)

register_product_button = tk.Button(register_product_frame, text='Add Product', command=add_product)
register_product_button.grid(row=0, column=2, padx=10, pady=10)

#Tạo các Widget cho trang truy vấn sản phẩm
query_combobox = ttk.Combobox(query_frame, width=35, state='readonly')
query_combobox['value'] = ('LIST ALL PRODUCT', 'LIST PRODUCT THAT ARE OUT OF STOCK', 'LIST PRODUCT THAT ARE IN STOCK', 'LIST SALE BY DATE')
query_combobox.grid(row=0, column=3, padx=10, pady=10)

query_button = tk.Button(query_frame, text='Select Query Type', command=query_inventory)
query_button.grid(row=0, columnspan=2, padx=10, pady=10)

query_listbox = tk.Listbox(query_frame, width=40, height=20)
query_listbox.grid(row=2, columnspan=2, padx=10, pady=10)

show_product()

window.mainloop()
